## DB Init

### 如何在設定好 table & column 後將資料塞進去

資料來源：[](https://ptx.transportdata.tw/PTX/)

In [1]:
import requests
from sqlalchemy import create_engine
import pandas as pd

In [2]:
header = {
    "accept": "application/json",
    "authorization": "hmac username=\"FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF\", algorithm=\"hmac-sha1\", headers=\"x-date\", signature=\"zB7PJevm26xiszJ4NYbwcuKbIss=\"",
    "sec-ch-ua": "\"Chromium\";v=\"94\", \"Google Chrome\";v=\"94\", \";Not A Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "x-date": "Sat, 16 Oct 2021 16:46:25 GMT"
}
r = requests.get("https://ptx.transportdata.tw/MOTC/v2/Rail/THSR/Station?$format=JSON", headers = header)
print(r.status_code)
Station = r.json()

200


In [3]:
header = {
    "accept": "application/json",
    "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja-JP;q=0.6,ja;q=0.5",
    "authorization": "hmac username=\"FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF\", algorithm=\"hmac-sha1\", headers=\"x-date\", signature=\"W000TmemiIhiJGxovsBXeQKFU8s=\"",
    "sec-ch-ua": "\"Chromium\";v=\"94\", \"Google Chrome\";v=\"94\", \";Not A Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-date": "Sat, 16 Oct 2021 16:47:02 GMT"
}
r = requests.get("https://ptx.transportdata.tw/MOTC/v2/Rail/THSR/ODFare?$format=JSON", headers = header)
print(r.status_code)
ODFare = r.json()

200


In [4]:
header = {
    "accept": "application/json",
    "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja-JP;q=0.6,ja;q=0.5",
    "authorization": "hmac username=\"FFFFFFFF-FFFF-FFFF-FFFF-FFFFFFFFFFFF\", algorithm=\"hmac-sha1\", headers=\"x-date\", signature=\"sSNKBe0n0Cf9jFPUARXkUKDIUMU=\"",
    "sec-ch-ua": "\"Chromium\";v=\"94\", \"Google Chrome\";v=\"94\", \";Not A Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-date": "Sat, 16 Oct 2021 16:47:53 GMT"
}
r = requests.get("https://ptx.transportdata.tw/MOTC/v2/Rail/THSR/GeneralTimetable?$format=JSON", headers = header)
print(r.status_code)
GeneralTimetable = r.json()
tempGeneralTimetable = pd.DataFrame(GeneralTimetable)['GeneralTimetable']


200


In [6]:
tempGeneralTimetable = pd.DataFrame(GeneralTimetable)['GeneralTimetable']

# table: station
station = pd.DataFrame(Station).filter(items=['StationID', 'StationName'])
for i in range(len(station)):
    station['StationName'][i] = station['StationName'][i]['Zh_tw']
station = station.rename(columns={"StationID": "station_id", "StationName": "station_name"})

# table: train
train = pd.DataFrame()
train.insert(0, 'train_id', [row['GeneralTrainInfo']['TrainNo'] for row in tempGeneralTimetable])
train.insert(1, 'starting_station_id', [row['GeneralTrainInfo']['StartingStationID'] for row in tempGeneralTimetable])
train.insert(2, 'ending_station_id', [row['GeneralTrainInfo']['EndingStationID'] for row in tempGeneralTimetable])
train.insert(3, 'mon', [True if row['ServiceDay']['Monday'] else False for row in tempGeneralTimetable])
train.insert(4, 'tue', [True if row['ServiceDay']['Tuesday'] else False for row in tempGeneralTimetable])
train.insert(5, 'wed', [True if row['ServiceDay']['Wednesday'] else False for row in tempGeneralTimetable])
train.insert(6, 'thu', [True if row['ServiceDay']['Thursday'] else False for row in tempGeneralTimetable])
train.insert(7, 'fri', [True if row['ServiceDay']['Friday'] else False for row in tempGeneralTimetable])
train.insert(8, 'sat', [True if row['ServiceDay']['Saturday'] else False for row in tempGeneralTimetable])
train.insert(9, 'sun', [True if row['ServiceDay']['Sunday'] else False for row in tempGeneralTimetable])

# table: schedule
temp = []
for row in tempGeneralTimetable:
    for row_ in row['StopTimes']:
        temp.append({'train_id': row['GeneralTrainInfo']['TrainNo'], 
                     'stop_sequence': row_['StopSequence'], 
                     'station_id': row_['StationID'], 
                     'arrival_time': row_['ArrivalTime'] if 'ArrivalTime' in row_ else "",
                     'departure_time': row_['DepartureTime'] if 'DepartureTime' in row_ else ""})
schedule = pd.DataFrame()
schedule.insert(0, 'train_id', [row['train_id'] for row in temp])
schedule.insert(1, 'stop_sequence', [row['stop_sequence'] for row in temp])
schedule.insert(2, 'station_id', [row['station_id'] for row in temp])
schedule.insert(3, 'arrival_time', [row['arrival_time'] for row in temp])
schedule.insert(4, 'departure_time', [row['departure_time'] for row in temp])

# table: fare
fare = pd.DataFrame(ODFare).filter(items=['OriginStationID', 'DestinationStationID'])
fare.insert(2, 'business_fare', [row[0]['Price'] for row in pd.DataFrame(ODFare)['Fares']])
fare.insert(3, 'standard_fare', [row[1]['Price'] for row in pd.DataFrame(ODFare)['Fares']])
fare.insert(4, 'non_reserved_fare', [row[2]['Price'] for row in pd.DataFrame(ODFare)['Fares']])
fare = fare.rename(columns={"OriginStationID": "origin_station_id", "DestinationStationID": "destination_station_id"})

# table: discount
temp = {'type': ['concession', 'group', 'early_bird', 'undergraduate'],
        'discount_rate': [0.5, 0.95, 0.8, 0.75]}
discount = pd.DataFrame(temp)

# fill in your db account info
username = input("Username:")
password = input("Password:")

# let's insert data
engine = create_engine("postgresql://" + username + ":" + password + "@localhost:5432/THSR")
station.to_sql('station', engine, schema = 'THSR', if_exists="append", index = False)
train.to_sql('train', engine, schema = 'THSR', if_exists="append", index = False)
schedule.to_sql('schedule', engine, schema = 'THSR', if_exists="append", index = False)
fare.to_sql('fare', engine, schema = 'THSR', if_exists="append", index = False)
discount.to_sql('discount', engine, schema = 'THSR', if_exists="append", index = False)

### 當你害怕會不小心把資料庫刪掉

你應該先備份資料庫，這裡提供兩種方法

- custom way
    - 這個方式適合在 GUI 還原
    - 新建一張同名的 schema 後，按右鍵 restore
- plain text way
    - 這個方式適合在 termial 用指令還原
    - `psql -U your_user_name your_db_name < your_dump_file`

